In [1]:
import pandas as pd
import numpy as np
import requests
import json
import matplotlib.pyplot as plt

**Trafik Verisi - Eksik Veri Tamamlama**

In [2]:
TrafikDataDF=pd.read_csv('IstanbulTraficData.csv',sep=';')
print("Veri Sayısı : "+str(TrafikDataDF["TrafficIndex"].count()))

Veri Sayısı : 127036


In [3]:
TrafikDataDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 127036 entries, 0 to 127035
Data columns (total 11 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   TrafficIndex      127036 non-null  int64 
 1   TrafficIndexDate  127036 non-null  object
 2   Date              127036 non-null  object
 3   Time              127036 non-null  object
 4   Day               127036 non-null  int64 
 5   Month             127036 non-null  int64 
 6   Year              127036 non-null  int64 
 7   Week              127036 non-null  int64 
 8   Days_Of_Week      127036 non-null  int64 
 9   Day_Name          127036 non-null  object
 10  Is_Weekend_Flag   127036 non-null  bool  
dtypes: bool(1), int64(6), object(4)
memory usage: 9.8+ MB


In [4]:
#Eksik Gün Hesaplama
TrafikDataDF['TrafficIndexDate']=TrafikDataDF['TrafficIndexDate'].apply(pd.to_datetime)
Days=pd.to_datetime("today").date()-TrafikDataDF.TrafficIndexDate.dt.date.max()
Days=Days.days
print("Eksik Gün Sayısı : "+str(Days))

Eksik Gün Sayısı : 0


In [5]:
#Eksik Saat Hesaplama
if (pd.to_datetime("today").time()==TrafikDataDF.loc[TrafikDataDF['TrafficIndexDate'].dt.date==TrafikDataDF.TrafficIndexDate.dt.date.max()].TrafficIndexDate.dt.time.max().strftime("%H:%M"))==False:
    Days=Days+1
    print("Değer Güncellendi. Yeni Değer : "+str(Days))

Değer Güncellendi. Yeni Değer : 1


In [6]:
#IBB veri setinden son 910 günün verisini 30 Dakika aralıklı olarak çekiyoruz
if Days!=0:
    response = requests.get("https://api.ibb.gov.tr/tkmservices/api/TrafficData/v1/TrafficIndexHistory/"+str(Days)+"/5M")
    TrafikDataDF=pd.concat([TrafikDataDF,pd.DataFrame(response.json())])
    TrafikDataDF=TrafikDataDF.drop_duplicates(subset=['TrafficIndexDate'])
    print("Veri Güncellendi. Yeni Veri Sayısı : "+str(TrafikDataDF["TrafficIndex"].count()))

Veri Güncellendi. Yeni Veri Sayısı : 98843


In [7]:
TrafikDataDF['TrafficIndexDate']=TrafikDataDF['TrafficIndexDate'].apply(pd.to_datetime)

TrafikDataDF['Date'] = TrafikDataDF.TrafficIndexDate.dt.date
TrafikDataDF['Time'] = TrafikDataDF.TrafficIndexDate.dt.time
TrafikDataDF["Day"]=TrafikDataDF.TrafficIndexDate.dt.day
TrafikDataDF["Month"]=TrafikDataDF.TrafficIndexDate.dt.month
TrafikDataDF["Year"]=TrafikDataDF.TrafficIndexDate.dt.year
TrafikDataDF["Week"]=TrafikDataDF.TrafficIndexDate.dt.week
TrafikDataDF['Days_Of_Week'] = TrafikDataDF['TrafficIndexDate'].dt.dayofweek
TrafikDataDF["Day_Name"]=TrafikDataDF.TrafficIndexDate.dt.day_name()
TrafikDataDF['Is_Weekend_Flag'] = np.where(TrafikDataDF['Day_Name'].isin(['Sunday', 'Saturday']), "True","False")

In [8]:
TrafikDataDF.count()

TrafficIndex        98843
TrafficIndexDate    98843
Date                98843
Time                98843
Day                 98843
Month               98843
Year                98843
Week                98843
Days_Of_Week        98843
Day_Name            98843
Is_Weekend_Flag     98843
dtype: int64

In [9]:
TrafikDataDF.to_csv('IstanbulTraficData.csv',sep=';',index=False)

**Araç Takip**

In [ ]:
docs=['https://data.ibb.gov.tr/dataset/ed3160cd-1756-40c7-9063-c3306ba71b06/resource/15e1332d-e476-4899-b3b9-8574ac8f6eb8/download/gunluk-arac-saym.csv','https://data.ibb.gov.tr/dataset/ed3160cd-1756-40c7-9063-c3306ba71b06/resource/b08038fd-b087-489f-910f-7c0aaf848325/download/gunluk-arac-saym.csv']

In [ ]:
for doc in docs:
    if doc==docs[0]:
        AracDataDF=pd.read_csv(doc,sep=';',encoding='iso-8859-1',dtype={"Arac Sayisi":str})
    else:
        AracDataDF=pd.concat([AracDataDF,pd.read_csv(doc,sep=';',encoding='iso-8859-1',dtype={"Arac Sayisi":str})])

In [ ]:
AracDataDF["Kordinat"]= AracDataDF["X Koordinati"].str.cat(AracDataDF["Y Koordinati"], sep =",")

In [ ]:
AracDataDF=AracDataDF.dropna(subset=['Arac Sayisi'])

In [ ]:
AracDataDF['Tarih']=AracDataDF['Tarih'].apply(pd.to_datetime)

AracDataDF["Day"]=AracDataDF.Tarih.dt.day
AracDataDF["Month"]=AracDataDF.Tarih.dt.month
AracDataDF["Year"]=AracDataDF.Tarih.dt.year
AracDataDF["Week"]=AracDataDF.Tarih.dt.week
AracDataDF['Days_Of_Week'] = AracDataDF['Tarih'].dt.dayofweek
AracDataDF["Day_Name"]=AracDataDF.Tarih.dt.day_name()
AracDataDF['Is_Weekend_Flag'] = np.where(AracDataDF['Day_Name'].isin(['Sunday', 'Saturday']), "True","False")

In [ ]:
AracDataDF.info()

In [ ]:
AracDataDF.to_csv('İstanbulAracData.csv',sep=';',index=False)